In [100]:
import os
import pandas as pd
start_directory = "20news-19997/20_newsgroups/"

articles_list = []
# Walk through all directories and files starting from the start_directory
for root, dirs, files in os.walk(start_directory):
    folder = root.replace(start_directory , "")
    if folder == "":
        continue
    for file_name in files:
        file_path = os.path.join(root, file_name)
        with open(file_path, "r", encoding="latin1") as file:
            file_contents = file.read()
            row_list = [file_contents, folder]
            articles_list.append(row_list)
            
articles = pd.DataFrame(articles_list, columns=['Articles', 'Group'])

articles

,Articles,Group
0,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49...,atheism
1,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,atheism
2,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,atheism
3,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,atheism
...,...,...
19992,Newsgroups: comp.sys.mac.hardware\nPath: canta...,sys.mac.hardware
19993,Newsgroups: comp.sys.mac.hardware\nPath: canta...,sys.mac.hardware
19994,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,sys.mac.hardware
19995,Newsgroups: comp.sys.mac.hardware\nPath: canta...,sys.mac.hardware
